In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
import time

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

################################
##### Transforms & Dataset #####
################################

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  
])

train_dataset = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

#######################
##### Dataloaders #####
#######################

BATCH_SIZE = 64

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True
)

test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False
)

########################
##### Baseline CNN #####
########################

class BaselineCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1), # 28×28 -> 28×28
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 14×14

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1), # 14×14
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # 7×7
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = BaselineCNN().to(DEVICE)

############################
##### Loss & optimizer #####
############################

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

#########################################
##### Training & evaluate functions #####
#########################################

def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    return running_loss / len(loader.dataset)


@torch.no_grad()
def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * images.size(0)

        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()

    avg_loss = running_loss / len(loader.dataset)
    accuracy = 100.0 * correct / len(loader.dataset)
    return avg_loss, accuracy

#########################
##### Training loop #####
#########################
EPOCHS = 20
history = []

for epoch in range(1, EPOCHS + 1):
    start = time.time()

    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE)
    val_loss, val_acc = evaluate(model, test_loader, criterion, DEVICE)

    elapsed = time.time() - start

    print(f"Epoch {epoch}")
    print(f" Train Loss: {train_loss:.4f}")
    print(f" Val Loss: {val_loss:.4f}")
    print(f" Val Acc: {val_acc:.2f}%")
    print(f" Time: {elapsed:.1f}s\n")

    history.append({
        "epoch": epoch,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "val_acc": val_acc
    })

### save history CSV
df = pd.DataFrame(history)
df.to_csv("fashion_mnist_baseline_cnn_history.csv", index=False)



Using device: cpu
Epoch 1
 Train Loss: 0.4966
 Val Loss: 0.3280
 Val Acc: 87.96%
 Time: 59.7s

Epoch 2
 Train Loss: 0.3200
 Val Loss: 0.3184
 Val Acc: 88.33%
 Time: 63.5s

Epoch 3
 Train Loss: 0.2700
 Val Loss: 0.2630
 Val Acc: 90.50%
 Time: 62.7s

Epoch 4
 Train Loss: 0.2385
 Val Loss: 0.2688
 Val Acc: 90.38%
 Time: 62.7s

Epoch 5
 Train Loss: 0.2135
 Val Loss: 0.2413
 Val Acc: 91.34%
 Time: 62.1s

Epoch 6
 Train Loss: 0.1939
 Val Loss: 0.2480
 Val Acc: 91.24%
 Time: 58.4s

Epoch 7
 Train Loss: 0.1768
 Val Loss: 0.2325
 Val Acc: 91.83%
 Time: 57.0s

Epoch 8
 Train Loss: 0.1620
 Val Loss: 0.2304
 Val Acc: 92.19%
 Time: 56.0s

Epoch 9
 Train Loss: 0.1492
 Val Loss: 0.2442
 Val Acc: 91.99%
 Time: 53.7s

Epoch 10
 Train Loss: 0.1371
 Val Loss: 0.2411
 Val Acc: 92.02%
 Time: 53.1s

Epoch 11
 Train Loss: 0.1257
 Val Loss: 0.2578
 Val Acc: 92.06%
 Time: 80.6s

Epoch 12
 Train Loss: 0.1166
 Val Loss: 0.2553
 Val Acc: 92.30%
 Time: 66.0s

Epoch 13
 Train Loss: 0.1075
 Val Loss: 0.2824
 Val Acc